<a href="https://colab.research.google.com/github/ArindamBanerji/wip-experiments/blob/master/misc/Combined_transformer_scaffolding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cobined Transformer Colab Scaffolding

<div style="background-color: #f0f0f0; padding: 20px; border-radius: 5px;">

## Overview
This notebook sets up and executes the time series forecasting application in Google Colab. It handles environment setup, module importing, and execution of the main forecasting code.

## Key Components

1. **Environment Setup**: Installs required Python libraries and checks GPU availability.

2. **Configuration**: Sets variables for directories, file names, version number, and batch size.

3. **Directory Navigation**: Ensures the notebook operates in the correct directory.

4. **File Check**: Verifies the existence of the main Python file.

5. **Execution Function**: `invoke_ViT_TS_args()`
   - Imports the main module
   - Sets up command-line arguments
   - Executes the main forecasting function

6. **Execution Trigger**: Calls the execution function if `parse_args` is True.

## Parameters Passed to main()
The `invoke_ViT_TS_args` function passes the following parameters to the `main()` function:

- `batch_size`: Integer value for the batch size (default is 34).
- `version_num`: String representing the version number (e.g., 'v35').
- `args`: An argparse.Namespace object containing:
  - `ticker`: Stock ticker symbol (default is 'AAPL').
  - `start_date`: Start date for data (default is '2020-01-01').
  - `end_date`: End date for data (default is '2023-12-31').
  - `batch_size`: Same as the `batch_size` parameter above.

## Usage
- Run cells sequentially from top to bottom.
- The scaffolding will set up the environment, load the forecasting module, and execute it.

## Customization
- Modify `version_number`, `base_dir`, `fnm`, and `default_batch_sz` as needed.
- Adjust arguments in `parser.parse_args()` within `invoke_ViT_TS_args()` for different stocks or date ranges.

## Troubleshooting
- Check `base_dir` and `fnm` if the main file is not found.
- Ensure Colab runtime is set to use GPU for GPU-related issues.
- Verify all required libraries are installed if import errors occur.

<p style="font-style: italic;">Note: This scaffolding is specific to Google Colab and may need modifications for different environments.</p>

</div>


In [ ]:
# setup the environment
! pip install yfinance numpy pandas matplotlib scikit-learn pyts torch einops vit-pytorch

In [ ]:
%matplotlib inline

In [ ]:
!nvidia-smi

In [ ]:
# set some variables + make sure to use GPU
root_dir = "/content/" #root dir that coalb uses
version_number = 'v35' # use a output version number
base_dir = "/content/drive/MyDrive/python-projects/kaggle_experiments/Transformer-TS/v3/" # local directory for execution
fnm = "combined_forecasting_coordinator.py" # name of python file from repo
default_batch_sz = 34 # set batch size - not being used
parse_args = True # not needed but code allows simple invocation

In [ ]:
# go to the root directory - helps for the git pulls
import os
os.chdir(root_dir)

In [ ]:
# change directory into the local repo
import os

cwd = os.getcwd()
if (os.path.samefile(cwd, base_dir) == False) :
    os.chdir(base_dir)

In [ ]:
#basic test to make sure that the file exists - should probably exit code
! (test -f "$fnm" && echo "file-exists") || echo "file-not-exists"

In [ ]:
# Option #2: dynamic import and call the main entry point.- do arg parse
import sys
import os
import importlib
import traceback  # Import traceback for error handling
import argparse  # Import argparse for command-line argument parsing

# does the heavy lifting of the call to vision-transformer code
def invoke_ViT_TS_args (default_batch_sz, base_dir, fnm, version_num) :

    print ("batch_sz", default_batch_sz)

    # check for fike path
    file_exists = os.path.exists(os.path.join(base_dir, fnm ))


    if not file_exists:
        print(f"Error: File for time series forecasting not found in {base_dir}")
    else:
        # Append the directory to Python's search path
        sys.path.append(base_dir)
        # get the module name
        module_name = fnm.replace('.py', '')

        # Attempt to import the module directly (no relative import)
        try:
            module = importlib.import_module(module_name)

            # Define command-line arguments
            parser = argparse.ArgumentParser()
            parser.add_argument('--ticker', type=str, required=True, help='Stock ticker symbol')
            parser.add_argument('--start_date', type=str, required=True, help='Start date for data')
            parser.add_argument('--end_date', type=str, required=True, help='End date for data')
            # Add an argument for batch size
            parser.add_argument('--batch_size', type=int, default=default_batch_sz, help='Batch size for training')
            args = parser.parse_args(['--ticker', 'AAPL', '--start_date', '2020-01-01', '--end_date', '2023-12-31', f'--batch_size={default_batch_sz}'])

            # Call the main function from the imported module, passing the batch size from args
            module.main(batch_size=args.batch_size, version_num=version_number, args=args)

        except SystemExit as e:
            print(f"The 'main' function exited with code: {e.code}")
            # Print the traceback to see what led to the SystemExit
            traceback.print_exc()

In [ ]:
# based on th parse_arg flag - do the call.
if parse_args :
      invoke_ViT_TS_args (default_batch_sz, base_dir, fnm, version_number)